In [6]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np

In [143]:
formations = {
    '4-3-3': ['GK','RB', 'CB', 'CB', 'LB', 'CDM', 'CM', 'CAM', 'RW', 'ST', 'LW'],
    '4-4-2': ['GK', 'RB', 'CB', 'CB', 'LB', 'RM', 'CM', 'CM', 'LM', 'ST', 'ST'],
    '4-2-3-1': ['GK', 'RB', 'CB', 'CB', 'LB', 'CDM', 'CDM', 'CAM', 'CAM', 'CAM', 'ST'],
    '4-3-2-1': ['GK','FB','FB','CB','CB','CM','CM','CM','OM','OM','CF'],
    '3-4-3' : ['GK','RCB','CB','LCB','RWB','M','M','LWB','RA','CA','LA'],
    '4-5-1' : ['GK','RB','CB','CB','LB','RM','RCM','DM','LCM','LM','S']
}
formations_name = list(formations.keys())
print(formations_name)
# print(formations[formations_name[0]])

['4-3-3', '4-4-2', '4-2-3-1', '4-3-2-1', '3-4-3', '4-5-1']


In [7]:
df = pd.read_csv("resluts_forq2.csv")
df.head()

,Unnamed: 0,jersey_number,nationality_name,name,birth,high,weight,sofifa_id,player_url,short_name,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,0,1,Denmark,Kasper Schmeichel,1986-11-05,185,76,163587,https://sofifa.com/player/163587/kasper-schmei...,K. Schmeichel,...,29+2,29+2,29+2,30+2,83+2,https://cdn.sofifa.net/players/163/587/22_120.png,https://cdn.sofifa.net/teams/95/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1331/60.png,https://cdn.sofifa.net/flags/dk.png
1,1,16,Denmark,Oliver Christensen,1999-03-22,190,82,236758,https://sofifa.com/player/236758/oliver-christ...,O. Christensen,...,25+2,25+2,25+2,24+2,69+2,https://cdn.sofifa.net/players/236/758/22_120.png,https://cdn.sofifa.net/teams/166/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/dk.png
2,2,22,Denmark,Frederik Roennow,1992-08-04,190,79,201269,https://sofifa.com/player/201269/frederik-ronn...,F. Rønnow,...,24+2,24+2,24+2,24+2,73+1,https://cdn.sofifa.net/players/201/269/22_120.png,https://cdn.sofifa.net/teams/1831/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1331/60.png,https://cdn.sofifa.net/flags/dk.png
3,3,2,Denmark,Joachim Andersen,1996-05-31,192,90,224221,https://sofifa.com/player/224221/joachim-ander...,J. Andersen,...,77+2,77+2,77+2,69+2,19+2,https://cdn.sofifa.net/players/224/221/22_120.png,https://cdn.sofifa.net/teams/1799/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1331/60.png,https://cdn.sofifa.net/flags/dk.png
4,4,4,Denmark,Simon Kjaer,1989-03-26,190,86,202276,https://sofifa.com/player/202276/simon-zoller/...,S. Zoller,...,49+2,49+2,49+2,54+2,19+2,https://cdn.sofifa.net/players/202/276/22_120.png,https://cdn.sofifa.net/teams/160/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/de.png


In [14]:
nations = set()
for item in df["nationality_name"]:
    nations.add(item)
nations = list(nations)
print(nations)


['Iran', 'Tunisia', 'Senegal', 'Switzerland', 'Denmark', 'Korea Republic', 'England', 'Australia', 'Ghana', 'Argentina', 'Benin', 'Spain', 'France', 'Poland', 'Cameroon', 'Morocco', 'Mexico', 'Netherlands', 'Serbia', 'Ecuador', 'Uruguay', 'Portugal', 'Germany', 'Croatia', 'Costa Rica', 'Wales', 'Saudi Arabia', 'Japan', 'Scotland', 'Brazil', 'Belgium']


In [138]:
def getBestFormation(nation):
    players = df.loc[df["nationality_name"]==nation].copy()
    players.sort_values(by="player_positions", inplace=True, ascending=True,key=lambda x:x.str.len())
    scores = []
    # for each formation
    for f_idx in range(len(formations_name)):
        positions = formations[formations_name[f_idx]]
        team = pd.DataFrame({
            "name" : "None",
            "position" : pd.Categorical(positions),
            "score" : 0
        })
        total_score=0
        for pos_idx in range(len(positions)):
            for player_idx in range(len(players)):
                if (positions[pos_idx] in players.iloc[player_idx]["player_positions"]) and (players.iloc[player_idx]["name"] not in team["name"].values):
                    if players.iloc[player_idx]["overall"]>team.iloc[pos_idx]["score"]:
                        nametmp = str(players.iloc[player_idx]["name"])
                        scoretmp = int(players.iloc[player_idx]["overall"])
                        team.loc[pos_idx,"name"] = nametmp
                        team.loc[pos_idx,"score"] = scoretmp
            total_score+=team.iloc[pos_idx]["score"]
        scores.append(total_score)
        if total_score == max(scores):
            best_team=team.copy()
#         display(team)
    
    best_formation = formations_name[scores.index(max(scores))]
    print("Best formation for {} is {}\nTeam as follow:\n".format(nation,best_formation))
    display(best_team)
#     return best_formation
            

In [144]:
getBestFormation("Argentina")

Best formation for Argentina is 4-4-2
Team as follow:



,name,position,score
0,Emiliano Martinez,GK,84
1,Gonzalo Montiel,RB,78
2,Lisandro Martínez,CB,79
3,Germán Pezzella,CB,77
4,Marcos Acuña,LB,84
5,Ángel Correa,RM,83
6,Paulo Dybala,CM,87
7,Rodrigo De Paul,CM,82
8,Angel Di Maria,LM,68
9,Lionel Messi,ST,93


In [145]:
for nation in nations:
    getBestFormation(nation)

Best formation for Iran is 4-4-2
Team as follow:



,name,position,score
0,Milad Mohammadi,GK,73
1,Sadegh Moharrami,RB,71
2,Morteza Pouraliganji,CB,69
3,Hossein Hosseini,CB,67
4,Ehsan Hajsafi,LB,72
5,Shoja Khalilzadeh,RM,74
6,Vahid Amiri,CM,77
7,Ali Gholizadeh,CM,74
8,Ahmad Nourollahi,LM,68
9,Mehdi Taremi,ST,79


Best formation for Tunisia is 4-4-2
Team as follow:



,name,position,score
0,Aymen Dahmen,GK,66
1,Ali Abdi,RB,70
2,Yassine Meriah,CB,69
3,Mouez Hassen,CB,67
4,Slimane A.,LB,62
5,Aymen Balbouli,RM,73
6,Ellyes Skhiri,CM,77
7,Aïssa Laïdouni,CM,70
8,Nader Ghandri,LM,76
9,Wahbi Khazri,ST,76


Best formation for Senegal is 4-4-2
Team as follow:



...

,name,position,score
0,None,GK,0
1,None,RB,0
2,None,CB,0
3,None,CB,0
4,None,LB,0
5,None,RM,0
6,None,RCM,0
7,None,DM,0
8,None,LCM,0
9,None,LM,0


Best formation for Brazil is 4-2-3-1
Team as follow:



,name,position,score
0,Ribeiro E.,GK,72
1,Gabriel Jesus,RB,62
2,Thiago Silva,CB,85
3,Éder Militão,CB,82
4,None,LB,0
5,Fred,CDM,81
6,Bruno Guimarães,CDM,79
7,Neymar,CAM,91
8,Weverton,CAM,80
9,Daniel Alves,CAM,72


Best formation for Belgium is 4-3-3
Team as follow:



,name,position,score
0,Thibaut Courtois,GK,89
1,Timothy Castagne,RB,79
2,Toby Alderweireld,CB,83
3,Jan Vertonghen,CB,81
4,Arthur Theate,LB,69
5,Youri Tielemans,CDM,84
6,Kevin De Bruyne,CM,91
7,Dries Mertens,CAM,84
8,Jeremy Doku,RW,77
9,Romelu Lukaku,ST,88
